In [3]:
# %%
# --- CODE CELL 1: Imports + Config + Functions (functions first) ---

import os
import re
import json
import time
import datetime as dt
from pathlib import Path
from typing import Dict, Any, Optional, Tuple, List

import requests
import pandas as pd

from IPython.display import display, clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt


# =========================
# Config
# =========================

ALPHAVANTAGE_API_KEY = "OYA0CFIEESAINF1M"
AV_BASE_URL = "https://www.alphavantage.co/query"

DATA_DIR = Path("data")
SECURITY_MASTER_DIR = DATA_DIR / "security_master"
ETF_HOLDINGS_DIR = DATA_DIR / "etf_holdings"
OVERRIDES_DIR = DATA_DIR / "overrides"

SECURITY_MASTER_PARQUET = SECURITY_MASTER_DIR / "security_master.parquet"
SECURITY_MASTER_META = SECURITY_MASTER_DIR / "meta.json"
ASSET_OVERRIDES_CSV = OVERRIDES_DIR / "asset_overrides.csv"


# =========================
# Helpers
# =========================

def ensure_dirs():
    SECURITY_MASTER_DIR.mkdir(parents=True, exist_ok=True)
    ETF_HOLDINGS_DIR.mkdir(parents=True, exist_ok=True)
    OVERRIDES_DIR.mkdir(parents=True, exist_ok=True)

def now_ts() -> str:
    return dt.datetime.now().strftime("%Y%m%d_%H%M%S")

def today_str() -> str:
    return dt.date.today().isoformat()

def symbol_norm(symbol: str) -> str:
    """
    Normalize ticker symbols for joining across sources.

    Rules (simple + practical):
    - uppercase, strip
    - replace '.' and '/' with '-'
    - collapse consecutive '-' into one
    """
    if symbol is None:
        return ""
    s = str(symbol).upper().strip()
    s = s.replace(".", "-").replace("/", "-")
    s = re.sub(r"-{2,}", "-", s)
    return s

def safe_float(x) -> Optional[float]:
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return None
    try:
        if isinstance(x, str):
            x = x.replace("$", "").replace(",", "").strip()
        return float(x)
    except Exception:
        return None

def parse_market_cap(x) -> Optional[float]:
    return safe_float(x)

def parse_last_sale(x) -> Optional[float]:
    return safe_float(x)

def alpha_vantage_get(params: Dict[str, Any], timeout: int = 30) -> Dict[str, Any]:
    """
    Alpha Vantage GET wrapper.
    Notes:
    - AV sometimes returns "Note" (rate limit) or "Information" (errors).
    """
    params = dict(params)
    params["apikey"] = ALPHAVANTAGE_API_KEY
    r = requests.get(AV_BASE_URL, params=params, timeout=timeout)
    r.raise_for_status()
    data = r.json()

    if isinstance(data, dict) and ("Note" in data or "Information" in data or "Error Message" in data):
        raise RuntimeError(f"Alpha Vantage response indicates an issue: {data}")

    return data


# =========================
# Overrides (Stock vs ETF)
# =========================

def load_asset_overrides() -> pd.DataFrame:
    ensure_dirs()
    if ASSET_OVERRIDES_CSV.exists():
        df = pd.read_csv(ASSET_OVERRIDES_CSV)
        if "ticker_norm" not in df.columns and "ticker" in df.columns:
            df["ticker_norm"] = df["ticker"].map(symbol_norm)
        if "is_etf" not in df.columns:
            df["is_etf"] = False
        df["ticker_norm"] = df["ticker_norm"].astype(str).map(symbol_norm)
        df["is_etf"] = df["is_etf"].astype(bool)
        df = df.drop_duplicates(subset=["ticker_norm"], keep="last")
        return df[["ticker_norm", "is_etf"]]
    return pd.DataFrame(columns=["ticker_norm", "is_etf"])

def save_asset_overrides(overrides_df: pd.DataFrame) -> None:
    ensure_dirs()
    out = overrides_df.copy()
    if "ticker_norm" not in out.columns:
        raise ValueError("Overrides must include ticker_norm")
    if "is_etf" not in out.columns:
        raise ValueError("Overrides must include is_etf")
    out["ticker_norm"] = out["ticker_norm"].astype(str).map(symbol_norm)
    out["is_etf"] = out["is_etf"].astype(bool)
    out = out.drop_duplicates(subset=["ticker_norm"], keep="last")
    out.to_csv(ASSET_OVERRIDES_CSV, index=False)


# =========================
# Security Master (Nasdaq Screener CSV)
# =========================

REQUIRED_MASTER_COLS = {"Symbol", "Name", "Country", "Sector", "Industry"}

def load_security_master() -> pd.DataFrame:
    ensure_dirs()
    if SECURITY_MASTER_PARQUET.exists():
        df = pd.read_parquet(SECURITY_MASTER_PARQUET)
        if "symbol_norm" not in df.columns and "Symbol" in df.columns:
            df["symbol_norm"] = df["Symbol"].map(symbol_norm)
        return df
    return pd.DataFrame(columns=["Symbol","Name","Country","Sector","Industry","symbol_norm"])

def refresh_security_master_from_csv(csv_path: str) -> pd.DataFrame:
    """
    Reads the Nasdaq Stock Screener CSV and persists a cleaned version as parquet.
    Keeps the raw CSV with timestamp so users can roll back if needed.
    """
    ensure_dirs()
    raw_path = Path(csv_path)
    if not raw_path.exists():
        raise FileNotFoundError(f"CSV not found at: {csv_path}")

    raw_copy = SECURITY_MASTER_DIR / f"raw_{now_ts()}.csv"
    raw_copy.write_bytes(raw_path.read_bytes())

    df = pd.read_csv(raw_copy)

    missing = REQUIRED_MASTER_COLS - set(df.columns)
    if missing:
        raise ValueError(f"Security master CSV is missing required columns: {sorted(list(missing))}")

    keep_cols = [c for c in ["Symbol","Name","Last Sale","Market Cap","Country","IPO Year","Volume","Sector","Industry"] if c in df.columns]
    df = df[keep_cols].copy()

    df["Symbol"] = df["Symbol"].astype(str).str.strip()
    df["symbol_norm"] = df["Symbol"].map(symbol_norm)

    if "Last Sale" in df.columns:
        df["last_sale_num"] = df["Last Sale"].map(parse_last_sale)
    if "Market Cap" in df.columns:
        df["market_cap_num"] = df["Market Cap"].map(parse_market_cap)

    df = df[df["symbol_norm"] != ""].copy()

    if "market_cap_num" in df.columns:
        df = df.sort_values(["symbol_norm","market_cap_num"], ascending=[True, False])
    df = df.drop_duplicates(subset=["symbol_norm"], keep="first")

    df.to_parquet(SECURITY_MASTER_PARQUET, index=False)

    meta = {
        "uploaded_at": dt.datetime.now().isoformat(),
        "raw_copy": str(raw_copy),
        "row_count": int(len(df)),
        "columns": list(df.columns),
    }
    SECURITY_MASTER_META.write_text(json.dumps(meta, indent=2))

    return df


# =========================
# ETF Holdings Cache (Alpha Vantage ETF_PROFILE)
# =========================

def holdings_cache_path(etf_symbol: str, asof_date: Optional[str] = None) -> Path:
    ensure_dirs()
    etf = symbol_norm(etf_symbol)
    d = asof_date or today_str()
    folder = ETF_HOLDINGS_DIR / etf
    folder.mkdir(parents=True, exist_ok=True)
    return folder / f"holdings_{d}.parquet"

def load_cached_etf_holdings(etf_symbol: str, asof_date: Optional[str] = None) -> Optional[pd.DataFrame]:
    path = holdings_cache_path(etf_symbol, asof_date)
    if path.exists():
        return pd.read_parquet(path)
    return None

def refresh_etf_holdings(etf_symbol: str, asof_date: Optional[str] = None, sleep_seconds: float = 0.0) -> pd.DataFrame:
    """
    Fetches ETF holdings from Alpha Vantage ETF_PROFILE and caches a daily snapshot.
    """
    etf_symbol = symbol_norm(etf_symbol)
    d = asof_date or today_str()

    data = alpha_vantage_get({"function": "ETF_PROFILE", "symbol": etf_symbol})

    holdings = None
    for k in ["holdings", "Holdings", "constituents", "Constituents"]:
        if k in data and isinstance(data[k], list):
            holdings = data[k]
            break

    if holdings is None:
        for v in data.values():
            if isinstance(v, list) and len(v) > 0 and isinstance(v[0], dict):
                keys = {k.lower() for k in v[0].keys()}
                if any(k in keys for k in ["symbol","ticker","holding","asset","constituent"]) and any(k in keys for k in ["weight","allocation","percentage","pct"]):
                    holdings = v
                    break

    if holdings is None:
        raise ValueError(f"Could not locate holdings list in ETF_PROFILE response for {etf_symbol}. Keys: {list(data.keys())}")

    hdf = pd.DataFrame(holdings).copy()

    lower_cols = {c: c.lower() for c in hdf.columns}
    sym_candidates = [c for c in hdf.columns if lower_cols[c] in ["symbol","ticker","holding","asset","constituent"]]
    wt_candidates = [c for c in hdf.columns if any(x in lower_cols[c] for x in ["weight","allocation","percentage","pct"])]

    if not sym_candidates:
        sym_candidates = [c for c in hdf.columns if "sym" in lower_cols[c]]
    if not wt_candidates:
        wt_candidates = [c for c in hdf.columns if "weight" in lower_cols[c] or "alloc" in lower_cols[c] or "pct" in lower_cols[c]]

    if not sym_candidates or not wt_candidates:
        raise ValueError(f"Could not infer holdings symbol/weight columns for {etf_symbol}. Columns: {list(hdf.columns)}")

    sym_col = sym_candidates[0]
    wt_col = wt_candidates[0]

    out = pd.DataFrame({
        "etf_symbol": etf_symbol,
        "constituent_symbol_raw": hdf[sym_col].astype(str).str.strip(),
        "constituent_symbol_norm": hdf[sym_col].astype(str).map(symbol_norm),
        "weight_raw": hdf[wt_col],
    })

    out["weight"] = out["weight_raw"].map(safe_float)
    if out["weight"].notna().any():
        if out["weight"].dropna().median() > 1.0:
            out["weight"] = out["weight"] / 100.0

    out = out.dropna(subset=["constituent_symbol_norm","weight"])
    out = out[out["constituent_symbol_norm"] != ""].copy()

    path = holdings_cache_path(etf_symbol, d)
    out.to_parquet(path, index=False)

    if sleep_seconds > 0:
        time.sleep(sleep_seconds)

    return out


# =========================
# Portfolio normalization + look-through
# =========================

def normalize_portfolio_inputs(df: pd.DataFrame, total_portfolio_value: Optional[float] = None) -> pd.DataFrame:
    """
    Supported inputs:
      - ticker (required)
      - percent (optional; 0..1)
      - shares (optional)
      - price_per_share (optional)
      - dollars (optional)

    Compute position_value:
      - dollars if provided
      - else shares * price_per_share if available
      - else percent * total_portfolio_value if provided
    """
    df = df.copy()
    df.columns = [c.strip().lower() for c in df.columns]
    if "ticker" not in df.columns:
        raise ValueError("Portfolio input must contain a 'ticker' column.")

    df["ticker_raw"] = df["ticker"].astype(str)
    df["ticker_norm"] = df["ticker_raw"].map(symbol_norm)

    if "price_per_share" not in df.columns:
        df["price_per_share"] = pd.NA

    def compute_value(row):
        if pd.notna(row.get("dollars")):
            return float(row["dollars"])
        if pd.notna(row.get("shares")) and pd.notna(row.get("price_per_share")):
            return float(row["shares"]) * float(row["price_per_share"])
        if pd.notna(row.get("percent")):
            if total_portfolio_value is None:
                raise ValueError("total_portfolio_value must be provided if any row uses 'percent'.")
            return float(row["percent"]) * float(total_portfolio_value)
        return None

    df["position_value"] = df.apply(compute_value, axis=1)

    bad = df[df["position_value"].isna()]
    if len(bad) > 0:
        raise ValueError(
            "Some rows are missing enough info to compute dollars exposure. "
            "Provide dollars OR shares+price_per_share OR percent+total_portfolio_value.\n"
            f"{bad[['ticker_raw','percent','shares','price_per_share','dollars']].to_string(index=False)}"
        )

    # Keep is_etf if present
    keep = ["ticker_raw","ticker_norm","percent","shares","price_per_share","dollars","position_value"]
    if "is_etf" in df.columns:
        keep.append("is_etf")

    return df[keep]

def apply_etf_classification(portfolio_df: pd.DataFrame) -> pd.DataFrame:
    """
    Determines asset_type based on:
    1) explicit portfolio_df.is_etf if present, else
    2) saved overrides in data/overrides/asset_overrides.csv, else
    3) default False
    """
    df = portfolio_df.copy()
    overrides = load_asset_overrides()

    if "is_etf" not in df.columns:
        df["is_etf"] = pd.NA

    df = df.merge(overrides, on="ticker_norm", how="left", suffixes=("", "_override"))

    # priority: explicit -> override -> False
    df["is_etf_final"] = df["is_etf"]
    df.loc[df["is_etf_final"].isna(), "is_etf_final"] = df.loc[df["is_etf_final"].isna(), "is_etf_override"]
    df["is_etf_final"] = df["is_etf_final"].fillna(False).astype(bool)

    df["asset_type"] = df["is_etf_final"].map(lambda x: "ETF" if bool(x) else "Stock")

    drop_cols = [c for c in ["is_etf_override"] if c in df.columns]
    df = df.drop(columns=drop_cols)

    return df

def enrich_with_security_master(df: pd.DataFrame, master_df: pd.DataFrame, join_col: str) -> pd.DataFrame:
    master_cols = ["symbol_norm","Name","Country","Sector","Industry"]
    master_min = master_df[master_cols].drop_duplicates("symbol_norm") if len(master_df) else pd.DataFrame(columns=master_cols)
    out = df.merge(master_min, left_on=join_col, right_on="symbol_norm", how="left")
    out = out.drop(columns=["symbol_norm"])
    return out

def build_lookthrough_exposures(
    portfolio_df: pd.DataFrame,
    master_df: pd.DataFrame,
    refresh_missing_etf_holdings: bool = True,
    asof_date: Optional[str] = None,
    per_etf_sleep_seconds: float = 0.0
) -> pd.DataFrame:
    rows = []
    d = asof_date or today_str()

    for _, r in portfolio_df.iterrows():
        src = r["ticker_norm"]
        src_type = r["asset_type"]
        pv = float(r["position_value"])

        if src_type == "Stock":
            rows.append({
                "source_ticker_norm": src,
                "source_type": src_type,
                "underlying_symbol_norm": src,
                "exposure_value": pv,
            })
        else:
            cached = load_cached_etf_holdings(src, d)
            if cached is None and refresh_missing_etf_holdings:
                cached = refresh_etf_holdings(src, d, sleep_seconds=per_etf_sleep_seconds)

            if cached is None or len(cached) == 0:
                rows.append({
                    "source_ticker_norm": src,
                    "source_type": src_type,
                    "underlying_symbol_norm": None,
                    "exposure_value": pv,
                })
            else:
                for _, h in cached.iterrows():
                    rows.append({
                        "source_ticker_norm": src,
                        "source_type": src_type,
                        "underlying_symbol_norm": h["constituent_symbol_norm"],
                        "exposure_value": pv * float(h["weight"]),
                    })

    exp = pd.DataFrame(rows)

    exp = enrich_with_security_master(exp, master_df, join_col="underlying_symbol_norm")

    exp["company_name"] = exp.get("Name")
    exp["country"] = exp.get("Country")
    exp["sector"] = exp.get("Sector")
    exp["industry"] = exp.get("Industry")

    return exp

def build_slices(exposures: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    by_company = (
        exposures.dropna(subset=["underlying_symbol_norm"])
        .groupby(["underlying_symbol_norm","company_name"], dropna=False)
        .agg(total_exposure=("exposure_value","sum"))
        .sort_values("total_exposure", ascending=False)
        .reset_index()
    )

    by_sector = (
        exposures.groupby("sector", dropna=False)
        .agg(total_exposure=("exposure_value","sum"))
        .sort_values("total_exposure", ascending=False)
        .reset_index()
    )

    by_country = (
        exposures.groupby("country", dropna=False)
        .agg(total_exposure=("exposure_value","sum"))
        .sort_values("total_exposure", ascending=False)
        .reset_index()
    )

    by_source_vehicle = (
        exposures.groupby(["source_ticker_norm","source_type"], dropna=False)
        .agg(total_exposure=("exposure_value","sum"))
        .sort_values("total_exposure", ascending=False)
        .reset_index()
    )

    return by_company, by_sector, by_country, by_source_vehicle

def find_unknown_underlyings(exposures: pd.DataFrame) -> pd.DataFrame:
    """
    Helps you see which tickers didn't match the local security master.
    """
    df = exposures.copy()
    df = df.dropna(subset=["underlying_symbol_norm"])
    unknown = df[df["company_name"].isna()][["underlying_symbol_norm"]].drop_duplicates()
    unknown = unknown.sort_values("underlying_symbol_norm").reset_index(drop=True)
    return unknown

def plot_bar(df: pd.DataFrame, category_col: str, value_col: str, title: str, top_n: int = 15):
    if df is None or len(df) == 0:
        print(f"No data to plot for {title}")
        return
    dd = df.copy()
    dd = dd.dropna(subset=[category_col])
    dd = dd.sort_values(value_col, ascending=False).head(top_n)
    plt.figure()
    plt.bar(dd[category_col].astype(str), dd[value_col].astype(float))
    plt.xticks(rotation=45, ha="right")
    plt.title(title)
    plt.ylabel(value_col)
    plt.tight_layout()
    plt.show()

ensure_dirs()

Matplotlib is building the font cache; this may take a moment.


In [ ]:
# %%
# --- CODE CELL 2: App UI (ipywidgets) ---

# -------------------------
# UI State
# -------------------------
STATE = {
    "master": load_security_master(),
    "portfolio_raw": None,
    "portfolio_norm": None,
    "portfolio_classified": None,
    "exposures": None,
    "slices": None,
}

# -------------------------
# UI Widgets
# -------------------------

title = widgets.HTML("<h3>Portfolio Look-Through App</h3>")

instructions = widgets.HTML(
    """
    <div style="line-height:1.35;">
      <b>1) Load / Refresh Security Master (Nasdaq Stock Screener CSV)</b><br>
      Required columns: <code>Symbol, Name, Country, Sector, Industry</code>.<br>
      This is used for slicing by country/sector/industry without paid APIs.<br><br>

      <b>2) Load Portfolio</b><br>
      Required: <code>ticker</code><br>
      Provide ONE of:<br>
      - <code>dollars</code> OR<br>
      - <code>shares</code> + <code>price_per_share</code> OR<br>
      - <code>percent</code> (0..1) + a <code>total_portfolio_value</code> in the UI<br><br>

      Optional: <code>is_etf</code> (True/False). If not provided, you can set overrides below.
    </div>
    """
)

# Security master upload
sm_upload = widgets.FileUpload(accept=".csv", multiple=False, description="Upload Security Master CSV")
sm_refresh_btn = widgets.Button(description="Refresh Security Master", button_style="primary")
sm_status = widgets.Output()

# Portfolio input: upload OR paste
pf_upload = widgets.FileUpload(accept=".csv", multiple=False, description="Upload Portfolio CSV")
pf_template_btn = widgets.Button(description="Show Portfolio Template", button_style="")
pf_paste = widgets.Textarea(
    value="ticker,dollars,is_etf\nAAPL,5000,False\nQQQ,8000,True\n",
    placeholder="Paste CSV here (header row required)",
    description="Or paste:",
    layout=widgets.Layout(width="100%", height="120px")
)
pf_load_btn = widgets.Button(description="Load Portfolio", button_style="primary")
pf_status = widgets.Output()

total_value = widgets.FloatText(value=0.0, description="Total $ (if %):", layout=widgets.Layout(width="300px"))

# ETF caching / refresh controls
asof_date = widgets.Text(value=today_str(), description="As-of date:", layout=widgets.Layout(width="260px"))
refresh_missing = widgets.Checkbox(value=True, description="Fetch missing ETF holdings (if not cached)", indent=False)
per_etf_sleep = widgets.FloatSlider(value=0.0, min=0.0, max=3.0, step=0.25, description="Sleep/sec per ETF", readout_format=".2f")

run_btn = widgets.Button(description="Run Look-Through", button_style="success")
run_status = widgets.Output()

# Overrides UI
overrides_box = widgets.Output()
save_overrides_btn = widgets.Button(description="Save ETF Overrides", button_style="warning")

# Filters
filter_sector = widgets.Dropdown(options=["(All)"], value="(All)", description="Sector:", layout=widgets.Layout(width="360px"))
filter_country = widgets.Dropdown(options=["(All)"], value="(All)", description="Country:", layout=widgets.Layout(width="360px"))
apply_filters_btn = widgets.Button(description="Apply Filters", button_style="")
filters_status = widgets.Output()

# Output area
out = widgets.Output()


# -------------------------
# Helpers for UI callbacks
# -------------------------

def _bytes_to_temp_csv(upload_widget: widgets.FileUpload, prefix: str) -> str:
    if not upload_widget.value:
        raise ValueError("No file uploaded.")
    item = list(upload_widget.value.values())[0]
    content = item["content"]
    name = item.get("metadata", {}).get("name", f"{prefix}.csv")
    tmp_path = DATA_DIR / f"tmp_{prefix}_{now_ts()}_{name}"
    tmp_path.write_bytes(content)
    return str(tmp_path)

def _read_portfolio_from_upload_or_paste() -> pd.DataFrame:
    if pf_upload.value:
        path = _bytes_to_temp_csv(pf_upload, "portfolio")
        df = pd.read_csv(path)
        return df
    # paste CSV
    text = pf_paste.value.strip()
    if not text:
        raise ValueError("Paste area is empty and no portfolio CSV uploaded.")
    from io import StringIO
    df = pd.read_csv(StringIO(text))
    return df

def _render_sm_status():
    with sm_status:
        clear_output()
        master = STATE["master"]
        meta = None
        if SECURITY_MASTER_META.exists():
            try:
                meta = json.loads(SECURITY_MASTER_META.read_text())
            except Exception:
                meta = None

        print(f"Security master rows: {len(master)}")
        if meta:
            print(f"Last refreshed: {meta.get('uploaded_at')}")
            print(f"Raw copy: {meta.get('raw_copy')}")
            print(f"Columns: {meta.get('columns')}")
        display(master.head(5))

def _render_pf_status():
    with pf_status:
        clear_output()
        df = STATE["portfolio_classified"]
        if df is None or len(df) == 0:
            print("No portfolio loaded.")
            return
        print(f"Portfolio rows: {len(df)}")
        display(df)

def _render_overrides_editor():
    """
    Lightweight overrides editor: list tickers in current portfolio, with a checkbox per ticker.
    Saves to data/overrides/asset_overrides.csv
    """
    with overrides_box:
        clear_output()
        pf = STATE.get("portfolio_norm")
        if pf is None or len(pf) == 0:
            print("Load a portfolio first to edit overrides.")
            return

        current_overrides = load_asset_overrides().set_index("ticker_norm")["is_etf"].to_dict()
        rows = []
        widgets_map = {}

        tickers = sorted(pf["ticker_norm"].unique().tolist())

        for t in tickers:
            cb = widgets.Checkbox(value=bool(current_overrides.get(t, False)), description=t, indent=False)
            widgets_map[t] = cb
            rows.append(cb)

        overrides_box._widgets_map = widgets_map  # stash for save
        col = widgets.VBox(rows, layout=widgets.Layout(max_height="220px", overflow="auto", border="1px solid #ddd", padding="6px"))
        display(widgets.HTML("<b>ETF Overrides (applies when portfolio input lacks is_etf)</b>"))
        display(col)

def _update_filter_options(exposures: pd.DataFrame):
    sectors = sorted([s for s in exposures["sector"].dropna().unique().tolist() if str(s).strip() != ""])
    countries = sorted([c for c in exposures["country"].dropna().unique().tolist() if str(c).strip() != ""])
    filter_sector.options = ["(All)"] + sectors
    filter_country.options = ["(All)"] + countries
    filter_sector.value = "(All)"
    filter_country.value = "(All)"

def _apply_filters_to_exposures(exposures: pd.DataFrame) -> pd.DataFrame:
    df = exposures.copy()
    if filter_sector.value != "(All)":
        df = df[df["sector"] == filter_sector.value]
    if filter_country.value != "(All)":
        df = df[df["country"] == filter_country.value]
    return df

def _render_outputs(exposures: pd.DataFrame):
    with out:
        clear_output()

        if exposures is None or len(exposures) == 0:
            print("No exposures to display.")
            return

        by_company, by_sector, by_country, by_source = build_slices(exposures)
        unknown = find_unknown_underlyings(exposures)

        print("Top companies (look-through):")
        display(by_company.head(25))

        print("\nSector exposure:")
        display(by_sector)

        print("\nCountry exposure:")
        display(by_country)

        print("\nSource vehicles (direct vs ETF):")
        display(by_source)

        if len(unknown) > 0:
            print("\nTickers not found in your Security Master (consider refreshing the master):")
            display(unknown.head(50))

        # Charts
        plot_bar(by_sector, "sector", "total_exposure", "Exposure by Sector (Top 15)", top_n=15)
        plot_bar(by_country, "country", "total_exposure", "Exposure by Country (Top 15)", top_n=15)


# -------------------------
# Callbacks
# -------------------------

def on_sm_refresh(_):
    with sm_status:
        clear_output()
        print("Refreshing security master...")
    try:
        path = _bytes_to_temp_csv(sm_upload, "security_master")
        STATE["master"] = refresh_security_master_from_csv(path)
        _render_sm_status()
    except Exception as e:
        with sm_status:
            clear_output()
            print(f"Security master refresh failed: {e}")

def on_pf_template(_):
    with pf_status:
        clear_output()
        print("Portfolio CSV template (copy/paste into a CSV file):\n")
        print("ticker,dollars,is_etf")
        print("AAPL,5000,False")
        print("QQQ,8000,True")
        print("\nAlternative template using shares + price_per_share:\n")
        print("ticker,shares,price_per_share,is_etf")
        print("AAPL,20,190,False")
        print("QQQ,15,420,True")
        print("\nAlternative template using percent (0..1) + Total $ in UI:\n")
        print("ticker,percent,is_etf")
        print("AAPL,0.25,False")
        print("QQQ,0.40,True")

def on_pf_load(_):
    try:
        df = _read_portfolio_from_upload_or_paste()

        tv = float(total_value.value) if total_value.value else None
        # If any percent is present, require total value
        if "percent" in [c.lower() for c in df.columns] and tv in (None, 0.0):
            # only require if any non-null percent exists
            tmp = df.copy()
            tmp.columns = [c.strip().lower() for c in tmp.columns]
            if "percent" in tmp.columns and tmp["percent"].notna().any():
                raise ValueError("Your portfolio includes 'percent'. Set Total $ (if %) in the UI (must be > 0).")

        pf_norm = normalize_portfolio_inputs(df, total_portfolio_value=(tv if tv and tv > 0 else None))
        STATE["portfolio_raw"] = df
        STATE["portfolio_norm"] = pf_norm

        pf_classified = apply_etf_classification(pf_norm)

        # Also enrich direct holdings with security master for nicer display
        pf_classified = enrich_with_security_master(pf_classified, STATE["master"], join_col="ticker_norm")
        STATE["portfolio_classified"] = pf_classified

        _render_pf_status()
        _render_overrides_editor()

    except Exception as e:
        with pf_status:
            clear_output()
            print(f"Portfolio load failed: {e}")

def on_save_overrides(_):
    try:
        wmap = getattr(overrides_box, "_widgets_map", {})
        if not wmap:
            raise ValueError("No overrides editor available. Load a portfolio first.")
        rows = [{"ticker_norm": t, "is_etf": bool(cb.value)} for t, cb in wmap.items()]
        df = pd.DataFrame(rows)
        save_asset_overrides(df)
        with overrides_box:
            print("\nSaved overrides to:", ASSET_OVERRIDES_CSV)
    except Exception as e:
        with overrides_box:
            print(f"\nFailed to save overrides: {e}")

def on_run(_):
    with run_status:
        clear_output()
        print("Running look-through...")

    try:
        master = STATE["master"]
        if master is None or len(master) == 0:
            raise ValueError("Security master is empty. Upload/refresh the Nasdaq screener CSV first.")

        pf = STATE.get("portfolio_norm")
        if pf is None or len(pf) == 0:
            raise ValueError("No portfolio loaded. Upload or paste portfolio CSV, then click Load Portfolio.")

        # Re-apply classification in case overrides were saved/changed
        pf_classified = apply_etf_classification(pf)
        pf_classified = enrich_with_security_master(pf_classified, master, join_col="ticker_norm")
        STATE["portfolio_classified"] = pf_classified

        d = asof_date.value.strip() or today_str()
        exp = build_lookthrough_exposures(
            portfolio_df=pf_classified,
            master_df=master,
            refresh_missing_etf_holdings=bool(refresh_missing.value),
            asof_date=d,
            per_etf_sleep_seconds=float(per_etf_sleep.value),
        )

        STATE["exposures"] = exp
        _update_filter_options(exp)

        filtered = _apply_filters_to_exposures(exp)
        _render_outputs(filtered)

        with run_status:
            clear_output()
            print("Done.")

    except Exception as e:
        with run_status:
            clear_output()
            print(f"Run failed: {e}")

def on_apply_filters(_):
    try:
        exp = STATE.get("exposures")
        if exp is None or len(exp) == 0:
            with filters_status:
                clear_output()
                print("Nothing to filter yet. Run look-through first.")
            return
        filtered = _apply_filters_to_exposures(exp)
        _render_outputs(filtered)
        with filters_status:
            clear_output()
            print("Filters applied.")
    except Exception as e:
        with filters_status:
            clear_output()
            print(f"Filter failed: {e}")

# Wire callbacks
sm_refresh_btn.on_click(on_sm_refresh)
pf_template_btn.on_click(on_pf_template)
pf_load_btn.on_click(on_pf_load)
save_overrides_btn.on_click(on_save_overrides)
run_btn.on_click(on_run)
apply_filters_btn.on_click(on_apply_filters)


# -------------------------
# Layout
# -------------------------

left = widgets.VBox([
    widgets.HTML("<b>Step 1 — Security Master</b>"),
    sm_upload,
    sm_refresh_btn,
    sm_status,
    widgets.HTML("<hr>"),
    widgets.HTML("<b>Step 2 — Portfolio Input</b>"),
    pf_upload,
    pf_template_btn,
    total_value,
    pf_paste,
    pf_load_btn,
    pf_status,
])

right = widgets.VBox([
    widgets.HTML("<b>Step 3 — ETF Overrides</b>"),
    overrides_box,
    save_overrides_btn,
    widgets.HTML("<hr>"),
    widgets.HTML("<b>Step 4 — Run</b>"),
    asof_date,
    refresh_missing,
    per_etf_sleep,
    run_btn,
    run_status,
    widgets.HTML("<hr>"),
    widgets.HTML("<b>Step 5 — Slice / Dice Filters</b>"),
    widgets.HBox([filter_sector, filter_country]),
    apply_filters_btn,
    filters_status,
])

app = widgets.VBox([
    title,
    instructions,
    widgets.HBox([left, right], layout=widgets.Layout(gap="16px")),
    widgets.HTML("<hr>"),
    out
])

# Initial renders
_render_sm_status()
_render_overrides_editor()

display(app)